In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pickle


In [114]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import mean_squared_error 

In [50]:
pkl_file = open('./JoinedData.pkl','rb')
df = pickle.load(pkl_file)
pkl_file.close()


In [52]:
df.dropna(how='any',inplace=True)
df.drop(columns=['FlightDate','date','Origin','ArrTime','ArrDel15','CRSArrTime','NearestDepTime'],inplace=True)
df = df[df['ArrDelayMinutes'] > 0]
df = df.apply(pd.to_numeric)
X = df.loc[:, df.columns != 'ArrDelayMinutes']
y = df.loc[:, df.columns == 'ArrDelayMinutes']

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.25)

In [134]:
print(X_test.shape)
print(y_test.shape)

(175071, 20)
(175071, 1)


In [53]:
X_train.dtypes

Year             float64
Quarter          float64
Month            float64
DayofMonth       float64
CRSDepTime       float64
DepTime          float64
totalSnow_cm     float64
windspeedKmph      int64
DewPointF          int64
cloudcover         int64
precipMM         float64
pressure           int64
WindGustKmph       int64
visibility         int64
weatherCode        int64
tempF              int64
WindChillF         int64
winddirDegree      int64
humidity           int64
time               int64
dtype: object

# Linear Regression

In [116]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score

In [117]:
lin_reg = LinearRegression(n_jobs=4)

In [118]:
lin_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=4, normalize=False)

In [119]:
y_pred = lin_reg.predict(X_test)

In [120]:

print(r2_score(y_test,y_pred))

0.022703105247050437


In [121]:
mean_squared_error(y_test,y_pred)

3677.1751200030585

# XGBoost Regressor

In [122]:
import xgboost as xgb

In [158]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.9, num_parallel_tree=3,learning_rate = 0.5, max_depth = 10,alpha = 10, n_estimators = 10,random_state=42)


In [159]:
xg_reg.fit(X_train,y_train)

[12:29:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0,
             importance_type='gain', learning_rate=0.5, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, num_parallel_tree=3,
             objective='reg:linear', random_state=42, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, seed=None, silent=None, subsample=1,
             verbosity=1)

In [160]:
y_pred = xg_reg.predict(X_test)

In [161]:
r2_score(y_test,y_pred)

0.9062336575038107

In [162]:
mean_squared_error(y_test,y_pred)

352.8050314820997

In [163]:
pred = pd.DataFrame(y_pred)
pred

,0
0,15.926425
1,9.951717
2,9.044063
3,11.167842
4,14.331500
...,...
175066,21.871538
175067,18.344696
175068,13.020017
175069,104.613899


In [129]:
y_test

,ArrDelayMinutes
1348618,2.0
1295495,6.0
138719,16.0
765415,16.0
282962,22.0
...,...
1193462,4.0
995520,12.0
646094,6.0
585932,109.0
